<a href="https://colab.research.google.com/github/shambhavi1709/Emotion-Aware-Chat-Agent/blob/main/EMOTIONAL_RAG_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from datasets import load_dataset

dataset = load_dataset("go_emotions")
dataset

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

simplified/train-00000-of-00001.parquet:   0%|          | 0.00/2.77M [00:00<?, ?B/s]

simplified/validation-00000-of-00001.par(…):   0%|          | 0.00/350k [00:00<?, ?B/s]

simplified/test-00000-of-00001.parquet:   0%|          | 0.00/347k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/43410 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5426 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5427 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'labels', 'id'],
        num_rows: 43410
    })
    validation: Dataset({
        features: ['text', 'labels', 'id'],
        num_rows: 5426
    })
    test: Dataset({
        features: ['text', 'labels', 'id'],
        num_rows: 5427
    })
})

In [2]:
from datasets.fingerprint import random

label_names = dataset["train"].features["labels"].feature.names

def get_emotions(row):
  print("Text:", row['text'])
  print("Emotion:", label_names[row['labels'][0]])

n = random.randint(0, len(dataset['train']))
get_emotions(dataset['train'][n])

Text: You sure are outraged about some people making fun of [NAME]. Gee, that's easy!
Emotion: anger


In [3]:
from transformers import AutoTokenizer

model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [4]:
def tokenize(batch):
    tokenized = tokenizer(batch["text"], truncation=True)
    tokenized["labels"] = batch["labels"]   # keep labels
    return tokenized

tokenized_dataset = dataset.map(tokenize, batched=True)
tokenized_dataset

Map:   0%|          | 0/43410 [00:00<?, ? examples/s]

Map:   0%|          | 0/5426 [00:00<?, ? examples/s]

Map:   0%|          | 0/5427 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'labels', 'id', 'input_ids', 'attention_mask'],
        num_rows: 43410
    })
    validation: Dataset({
        features: ['text', 'labels', 'id', 'input_ids', 'attention_mask'],
        num_rows: 5426
    })
    test: Dataset({
        features: ['text', 'labels', 'id', 'input_ids', 'attention_mask'],
        num_rows: 5427
    })
})

In [5]:
num_labels = 28

In [6]:
import torch

def convert_labels_to_multi_hot(examples):
    batch_multi_hot_labels = []
    for labels_list in examples["labels"]:
        multi_hot_vector = [0.0] * num_labels
        for label_id in labels_list:
            if label_id < num_labels:
                multi_hot_vector[label_id] = 1.0
        batch_multi_hot_labels.append(multi_hot_vector)
    return {"labels": torch.tensor(batch_multi_hot_labels, dtype=torch.float32)}

# Apply the conversion to the entire tokenized_dataset
tokenized_dataset = tokenized_dataset.map(convert_labels_to_multi_hot, batched=True)

# Set the format of the dataset to PyTorch tensors and specify columns/types
tokenized_dataset.set_format(
    type='torch',
    columns=['input_ids', 'attention_mask', 'labels'],
    output_all_columns=False
)

Map:   0%|          | 0/43410 [00:00<?, ? examples/s]

Map:   0%|          | 0/5426 [00:00<?, ? examples/s]

Map:   0%|          | 0/5427 [00:00<?, ? examples/s]

In [7]:
from transformers import DataCollatorWithPadding

# Define a custom DataCollator to ensure labels are always float32
class CustomDataCollator(DataCollatorWithPadding):
    def __call__(self, features):
        # Call the base DataCollatorWithPadding to handle tokenized inputs
        batch = super().__call__(features)
        # Explicitly cast labels to float32 if they exist in the batch
        if "labels" in batch:
            batch["labels"] = batch["labels"].to(torch.float32)
        return batch

# Instantiate the custom data collator
data_collator = CustomDataCollator(tokenizer=tokenizer)

In [8]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
    problem_type="multi_label_classification"
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
import numpy as np
import torch
import evaluate

metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred

    # Convert logits → probabilities → binary predictions
    probs = torch.sigmoid(torch.tensor(logits))
    preds = (probs > 0.5).int().numpy()

    # Convert references to int
    labels = labels.astype(int)

    # Flatten for macro/micro averaging
    preds_flat = preds.reshape(-1)
    labels_flat = labels.reshape(-1)

    return {
        "f1_macro": metric.compute(
            predictions=preds_flat,
            references=labels_flat,
            average="macro",
        )["f1"],
        "f1_micro": metric.compute(
            predictions=preds_flat,
            references=labels_flat,
            average="micro",
        )["f1"],
    }


In [13]:
# !pip install evaluate
# import evaluate
# import numpy as np

# metric = evaluate.load("f1")

# def compute_metrics(eval_pred):
#     logits, labels = eval_pred
#     preds = (torch.sigmoid(torch.tensor(logits)) > 0.5).int().numpy()
#     # Cast references to int type, as required by the f1 metric for multi-label classification
#     references = labels.int().numpy()
#     return metric.compute(predictions=preds, references=references, average="macro")

In [17]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="emotion_model",
    eval_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    learning_rate=2e-5,
    load_best_model_at_end=True,
    report_to="none"
)

In [18]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()


/tmp/ipython-input-2024412654.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,F1 Macro,F1 Micro
1,0.058000,0.092457,0.779501,0.968202
2,0.060800,0.089711,0.780211,0.968636


TrainOutput(global_step=5428, training_loss=0.061649890005720305, metrics={'train_runtime': 385.9689, 'train_samples_per_second': 224.94, 'train_steps_per_second': 14.063, 'total_flos': 770469756330528.0, 'train_loss': 0.061649890005720305, 'epoch': 2.0})

In [19]:
trainer.save_model("emotion_classifier")
tokenizer.save_pretrained("emotion_classifier")


('emotion_classifier/tokenizer_config.json',
 'emotion_classifier/special_tokens_map.json',
 'emotion_classifier/vocab.txt',
 'emotion_classifier/added_tokens.json',
 'emotion_classifier/tokenizer.json')